# Equilibrate extension
Notebook to illustrate the calculation of forcing a silicate liquid to be in equilibrium with specified chemical potentials of oxygen and water, under conditions where neither phase is present in the system

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import scipy.linalg as lin 
%matplotlib notebook

### Define a function to calculate the Gibbs free energy of oxygen gas

In [ ]:
def gOxygen(t, p):
    tr = 298.15
    pr = 1.0
    hs = 23.10248*(t-tr) + 2.0*804.8876*(np.sqrt(t)-np.sqrt(tr)) \
       - 1762835.0*(1.0/t-1.0/tr) \
       - 18172.91960*np.log(t/tr) + 0.5*0.002676*(t*t-tr*tr)
    ss = 205.15 + 23.10248*np.log(t/tr) \
       - 2.0*804.8876*(1.0/np.sqrt(t)-1.0/np.sqrt(tr)) \
       - 0.5*1762835.0*(1.0/(t*t)-1.0/(tr*tr)) \
       + 18172.91960*(1.0/t-1.0/tr) + 0.002676*(t-tr) 
    vs = 8.3143*t/pr
    return hs - t*ss

### Get class instances to calculate the properties of silicate liquid from MELTS 1.0.2

In [ ]:
import ctypes
from ctypes import cdll
from ctypes import util
from rubicon.objc import ObjCClass, objc_method
cdll.LoadLibrary(util.find_library('phaseobjc'))

In [ ]:
LiquidMelts = ObjCClass('LiquidMelts')
liquid = LiquidMelts.alloc().init()
WaterMelts = ObjCClass('WaterMelts')
water = WaterMelts.alloc().init()

### Set number of components in the system and the number of fixed phases

In [ ]:
c = 4
f = 2

### Choose and initial bulk composition for the silicate liquid 
The initial composition listed below is calculated for the early erupted Bishop Tuff

In [ ]:
nc = liquid.numberOfSolutionComponents()
bc = np.zeros((c,1))
nSiO2    = 0.665792
nTiO2    = 0.000600
nAl2O3   = 0.042436
nFe2O3   = 0.000777
nFe2SiO4 = 0.001973
nMg2SiO4 = 0.000223
nCaSiO3  = 0.004596
nNa2SiO3 = 0.038493
nKAlSiO4 = 0.062105
nH2O     = 0.183006
nFeTot   = 2.0*nFe2O3 + 2.0*nFe2SiO4
def setBulkComposition(nFe2O3in=0.000777, nH2Oin=0.183006):
    bc[0] = nSiO2 + nFe2SiO4      # SiO2
    bc[1] = nFe2O3in              # Fe2O3
    bc[2] = nFeTot - 2.0*nFe2O3in # FeO
    bc[3] = nH2Oin                # H2O
m = (ctypes.c_double*nc)()
ctypes.cast(m, ctypes.POINTER(ctypes.c_double))
def setMoles(nFe2O3in=0.000777, nH2Oin=0.183006):
    exSiO2 = nFe2SiO4 - (nFeTot - 2.0*nFe2O3in)
    nLiq = np.zeros((c,1))
    nLiq[0] = nSiO2 + nFe2SiO4 + exSiO2
    nLiq[1] = nFe2O3in
    nLiq[2] = (nFeTot - 2.0*nFe2O3in)/2.0
    nLiq[3] = nH2Oin
    m[0] = nLiq[0]  # SiO2
    m[1] = 0.0      # TiO2
    m[2] = 0.0      # Al2O3
    m[3] = nLiq[1]  # Fe2O3
    m[4] = 0.0      # Cr2O3
    m[5] = nLiq[2]  # Fe2SiO4
    m[6] = 0.0      # Mn2SiO4
    m[7] = 0.0      # Mg2SiO4
    m[8] = 0.0      # NiSi1/2O2
    m[9] = 0.0      # CoSi1/2O2
    m[10] = 0.0     # CaSiO3
    m[11] = 0.0     # Na2SiO3
    m[12] = 0.0     # KAlSiO4
    m[13] = 0.0     # Ca3(PO4)2
    m[14] = nLiq[3] # H2O
    return nLiq

### Choose a temperature and pressure
The thermodynamic properties of O<sub>2</sub> and H<sub>2</sub>O are only functions of temperature and pressure

In [ ]:
t = 1053.15 # K
p = 1750.0  # bars
mu0O2 = gOxygen(t, p)
mu0H2O = water.getGibbsFreeEnergyFromT_andP_(t, p)
muO2 = mu0O2 + (-24930.0/t +  9.360)*np.log(10.0)*8.3143*t # NNO
muH2O = mu0H2O + np.log(1.0)*8.3143*t                      # activity = 0.5

### Find an initial guess that is consistent with the constraints

In [ ]:
def gcon(x):
    t0 =  1673.15  # K
    a  =  0.196
    b  =  1.1492e4 # K
    c  = -6.675
    e  = -3.364
    f  = -7.01e-7  * 1.0e5 # K/bar
    g  = -1.54e-10 * 1.0e5 # 1/bar
    h =   3.85e-17 * 1.0e5 * 1.0e5 # K/bar^2
    temp = b/t + c + e*(1.0-t0/t - np.log(t/t0)) + f*p/t + g*(t-t0)*p/t+ h*p*p/t
    mSiO2  = nSiO2 + nFe2SiO4 + nMg2SiO4 + nCaSiO3 + nNa2SiO3 + nKAlSiO4
    mTiO2  = nTiO2
    mAl2O3 = nAl2O3
    mFeTot = nFeTot
    mMgO   = 2.0*nMg2SiO4
    mCaO   = nCaSiO3
    mNa2O  = nNa2SiO3
    mK2O   = nKAlSiO4
    mH2O   = x[1]
    total  = mSiO2 + mTiO2 + mAl2O3 + mFeTot + mMgO + mCaO + mNa2O + mK2O + mH2O
    dAl2O3 = -2.243
    dFeO   = -1.828
    dCaO   =  3.201
    dNa2O  =  5.854
    dK2O   =  6.215
    temp += dAl2O3*mAl2O3/total + dFeO*mFeTot/total + dCaO*mCaO/total + dNa2O*mNa2O/total + dK2O*mK2O/total
    mFe2O3 = x[0]
    mFeO   = mFeTot - 2.0*mFe2O3
    return 8.3143*t*(np.log(mFe2O3/mFeO) - temp)/a + mu0O2
def fcon(x):
    setMoles(nFe2O3in=x[0], nH2Oin=x[1])
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    if useKressCarmichael:
        zeroMuO2 = muO2 - gcon(x)
        #print (zeroMuO2, muO2)
    else:
        zeroMuO2 = muO2 + 2.0*muLiq.valueAtIndex_(5) - 2.0*muLiq.valueAtIndex_(0) - 2.0*muLiq.valueAtIndex_(3)
    zeroMuH2O = muH2O - muLiq.valueAtIndex_(14)
    return (zeroMuO2)**2 + (zeroMuH2O)**2
useKressCarmichael = 0
result = opt.minimize(fcon,np.array([0.000777, 0.183006]),bounds=((0,nFeTot/2.0),(0,None)))
print (result)
nLiqFe2O3 = result.x[0]
nLiqH2O = result.x[1]
setBulkComposition(nFe2O3in=nLiqFe2O3, nH2Oin=nLiqH2O)
print (bc)

### Set up constraint matrix - matrix is constant
$$
\left[ {\begin{array}{*{20}{c}}
{{b_{{\rm{Si}}{{\rm{O}}_{\rm{2}}}}}}\\
{{b_{{\rm{F}}{{\rm{e}}_{\rm{2}}}{{\rm{O}}_{\rm{3}}}}}}\\
{{b_{{\rm{FeO}}}}}\\
{{b_{{{\rm{H}}_{\rm{2}}}{\rm{O}}}}}
\end{array}} \right] = \left[ {\begin{array}{*{20}{c}}
1&0&1&0&0&0\\
0&1&0&0&2&0\\
0&0&2&0&{ - 4}&0\\
0&0&0&1&0&1\\
\end{array}} \right]\left[ {\begin{array}{*{20}{c}}
{n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}\\
{n_{{\rm{F}}{{\rm{e}}_2}{{\rm{O}}_3}}^{liq}}\\
{n_{{\rm{F}}{{\rm{e}}_2}{\rm{Si}}{{\rm{O}}_4}}^{liq}}\\
{n_{{{\rm{H}}_2}{\rm{O}}}^{liq}}\\
{n_{{{\rm{O}}_2}}^{sys}}\\
{n_{{{\rm{H}}_2}{\rm{O}}}^{sys}}
\end{array}} \right]
$$

In [ ]:
C = np.array([[1,0,1,0,0,0],[0,1,0,0,2,0],[0,0,2,0,-4,0],[0,0,0,1,0,1]])

### ...  and project it into the null space of fixed constraints - yields a constant matrix

In [ ]:
CTf = C[:,c:].transpose()
U,S,VT = np.linalg.svd(CTf)
rank = np.count_nonzero(S)
VTff = VT[rank:,:]
Cproj = np.matmul(VTff, C)
print ("Cproj")
for i in range(0,Cproj.shape[0]):
    print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e} {5:10.3e}".format( \
       Cproj[i][0], Cproj[i][1], Cproj[i][2], Cproj[i][3], Cproj[i][4], Cproj[i][5]))
bcproj = np.matmul(VTff, bc)
print ("bcproj")
for i in range(0,bcproj.shape[0]):
    print ("{0:10.3e}".format(bcproj[i][0]))

### Define the Khorzhinski potential function and its gradient
$$
L = n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq} + n_{{\rm{F}}{{\rm{e}}_2}{{\rm{O}}_3}}^{liq}\mu _{{\rm{F}}{{\rm{e}}_2}{{\rm{O}}_3}}^{liq} + n_{{\rm{F}}{{\rm{e}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_4}}^{liq}\mu _{{\rm{F}}{{\rm{e}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_4}}^{liq} + n_{{{\rm{H}}_2}{\rm{O}}}^{liq}\mu _{{{\rm{H}}_2}{\rm{O}}}^{liq} - n_{{{\rm{O}}_2}}^{sys}\mu _{{{\rm{O}}_2}}^{sys} - n_{{{\rm{H}}_2}{\rm{O}}}^{sys}\mu _{{{\rm{H}}_2}{\rm{O}}}^{sys}
$$

In [ ]:
def Khorzhinskii(nFe2O3in=0.000777, nH2Oin=0.183006, t=1053.15, p=1750.0):
    setMoles(nFe2O3in, nH2Oin) # fills m array
    Gliq = liquid.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m,t,p)
    result = Gliq - (2.0*m[3]+2.0*m[0]-2.0*m[5])*muO2 - m[14]*muH2O
    return result

### Define the gradient of the Khorzhinskii potential
$$
{\bf{g}} = \left[ {\begin{array}{*{20}{c}}
{\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}\\
{\mu _{{\rm{F}}{{\rm{e}}_2}{{\rm{O}}_3}}^{liq}}\\
{\mu _{{\rm{F}}{{\rm{e}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_4}}^{liq}}\\
{\mu _{{{\rm{H}}_2}{\rm{O}}}^{liq}}\\
{ - \mu _{{{\rm{O}}_2}}^{sys} - n_{{{\rm{O}}_2}}^{sys}\frac{{\partial \mu _{{{\rm{O}}_2}}^{sys}}}{{\partial n_{{{\rm{O}}_2}}^{sys}}}}\\
{ - \mu _{{{\rm{H}}_2}{\rm{O}}}^{sys} - n_{{{\rm{H}}_2}{\rm{O}}}^{sys}\frac{{\partial \mu _{{{\rm{H}}_2}{\rm{O}}}^{sys}}}{{\partial n_{{{\rm{H}}_2}{\rm{O}}}^{sys}}}}
\end{array}} \right]
$$

In [ ]:
def Gradient(nFe2O3in=0.000777, nH2Oin=0.183006, t=1053.15, p=1750.0):
    setMoles(nFe2O3in, nH2Oin) # fills m array
    dgdm = liquid.getDgDmFromMolesOfComponents_andT_andP_(m, t, p)
    result = np.zeros((c+f,1))
    result[0] = dgdm.valueAtIndex_(0)
    result[1] = dgdm.valueAtIndex_(3)
    result[2] = dgdm.valueAtIndex_(5)
    result[3] = dgdm.valueAtIndex_(14)
    result[4] = -muO2
    result[5] = -muH2O
    return result

### Define a function to compute the "A" constraint matrix
$$
{\bf{A}} = \left[ {\begin{array}{*{20}{c}}
0&{\frac{2}{{\sqrt 5 }}}&{\frac{2}{{\sqrt 5 }}}&0&0&0\\
1&0&1&0&0&0\\
{ - \frac{{\partial \mu _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \mu _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{liq}}}{{\partial n_{{\rm{F}}{{\rm{e}}_{\rm{2}}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{liq}}}{{\partial n_{{\rm{F}}{{\rm{e}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_4}}^{liq}}}}&{ - \frac{{\partial \mu _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{liq}}}{{\partial n_{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{liq}}}}&0&{\frac{{\partial \mu _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{sys}}}{{\partial n_{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{sys}}}}\\
{ - \frac{{\partial \mu _{{{\rm{O}}_{\rm{2}}}}^{liq}}}{{\partial n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}}&{ - \frac{{\partial \mu _{{{\rm{O}}_{\rm{2}}}}^{liq}}}{{\partial n_{{\rm{F}}{{\rm{e}}_{\rm{2}}}{{\rm{O}}_3}}^{liq}}}}&{ - \frac{{\partial \mu _{{{\rm{O}}_{\rm{2}}}}^{liq}}}{{\partial n_{{\rm{F}}{{\rm{e}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_4}}^{liq}}}}&{ - \frac{{\partial \mu _{{{\rm{O}}_{\rm{2}}}}^{liq}}}{{\partial n_{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{liq}}}}&{\frac{{\partial \mu _{{{\rm{O}}_{\rm{2}}}}^{sys}}}{{\partial n_{{{\rm{O}}_2}}^{sys}}}}&0\\
{ - 2}&{ - 2}&2&0&1&0\\
0&0&0&{ - 1}&0&1
\end{array}} \right]
$$
$$
\frac{{\partial \mu _{{{\rm{O}}_2}}^{liq}}}{{\partial n_i^{liq}}} = 2\frac{{\partial \mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq}}}{{\partial n_i^{liq}}} + 2\frac{{\partial \mu _{{\rm{F}}{{\rm{e}}_{\rm{2}}}{{\rm{O}}_3}}^{liq}}}{{\partial n_i^{liq}}} - 2\frac{{\partial \mu _{{\rm{F}}{{\rm{e}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_4}}^{liq}}}{{\partial n_i^{liq}}}
$$

In [ ]:
def Amatrix(nFe2O3in=0.000777, nH2Oin=0.183006, t=1053.15, p=1750.0):
    setMoles(nFe2O3in, nH2Oin) # fills m array
    d2gdm2 = liquid.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
    bottom = np.zeros((2*f,c+f))
    bottom[0][0] = -d2gdm2.valueAtRowIndex_andColIndex_(14, 0)
    bottom[0][1] = -d2gdm2.valueAtRowIndex_andColIndex_(14, 3)
    bottom[0][2] = -d2gdm2.valueAtRowIndex_andColIndex_(14, 5)
    bottom[0][3] = -d2gdm2.valueAtRowIndex_andColIndex_(14, 14)
    bottom[1][0] = -( 2.0*d2gdm2.valueAtRowIndex_andColIndex_(0, 0) \
                     +2.0*d2gdm2.valueAtRowIndex_andColIndex_(3, 0) \
                     -2.0*d2gdm2.valueAtRowIndex_andColIndex_(5, 0))
    bottom[1][1] = -( 2.0*d2gdm2.valueAtRowIndex_andColIndex_(0, 3) \
                     +2.0*d2gdm2.valueAtRowIndex_andColIndex_(3, 3) \
                     -2.0*d2gdm2.valueAtRowIndex_andColIndex_(5, 3))
    bottom[1][2] = -( 2.0*d2gdm2.valueAtRowIndex_andColIndex_(0, 5) \
                     +2.0*d2gdm2.valueAtRowIndex_andColIndex_(3, 5) \
                     -2.0*d2gdm2.valueAtRowIndex_andColIndex_(5, 5))
    bottom[1][3] = -( 2.0*d2gdm2.valueAtRowIndex_andColIndex_(0,14) \
                     +2.0*d2gdm2.valueAtRowIndex_andColIndex_(3,14) \
                     -2.0*d2gdm2.valueAtRowIndex_andColIndex_(5,14))
    bottom[2][0] = -2.0
    bottom[2][1] = -2.0
    bottom[2][2] =  2.0
    bottom[2][4] =  1.0
    bottom[3][3] = -1.0
    bottom[3][5] =  1.0
    result = np.vstack((Cproj, bottom))
    return result

### Define the Lagrangian of the Khorzhimskii function
$$
\begin{array}{c}
\Lambda  = n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}\mu _{{\rm{Si}}{{\rm{O}}_2}}^{liq} + n_{{\rm{F}}{{\rm{e}}_2}{{\rm{O}}_3}}^{liq}\mu _{{\rm{F}}{{\rm{e}}_2}{{\rm{O}}_3}}^{liq} + n_{{\rm{F}}{{\rm{e}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_4}}^{liq}\mu _{{\rm{F}}{{\rm{e}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_4}}^{liq} + n_{{{\rm{H}}_2}{\rm{O}}}^{liq}\mu _{{{\rm{H}}_2}{\rm{O}}}^{liq} - n_{{{\rm{O}}_2}}^{sys}\mu _{{{\rm{O}}_2}}^{sys} - n_{{{\rm{H}}_2}{\rm{O}}}^{sys}\mu _{{{\rm{H}}_2}{\rm{O}}}^{sys}\\
 - {\lambda _{{\rm{F}}{{\rm{e}}_{Tot}}}}\left( {\frac{2}{{\sqrt 5 }}n_{{\rm{F}}{{\rm{e}}_2}{{\rm{O}}_3}}^{liq} + \frac{2}{{\sqrt 5 }}n_{{\rm{F}}{{\rm{e}}_2}{\rm{Si}}{{\rm{O}}_4}}^{liq} - \frac{2}{{\sqrt 5 }}{b_{{\rm{F}}{{\rm{e}}_2}{{\rm{O}}_3}}} - \frac{1}{{\sqrt 5 }}n_{{\rm{FeO}}}^{liq}} \right) - {\lambda _{{\rm{Si}}{{\rm{O}}_2}}}\left( {n_{{\rm{Si}}{{\rm{O}}_2}}^{liq} - {b_{{\rm{Si}}{{\rm{O}}_2}}}} \right)\\
 - {\lambda _{{{\rm{H}}_{\rm{2}}}{\rm{O,}}\mu }}\left( {\mu _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{sys} - \mu _{{{\rm{H}}_{\rm{2}}}{\rm{O}}}^{liq}} \right) - {\lambda _{{{\rm{O}}_2},\mu }}\left( {\mu _{{{\rm{O}}_2}}^{sys} - \mu _{{{\rm{O}}_2}}^{liq}} \right)\\
- {\lambda _{{{\rm{O}}_2},n}}\left( {n_{{{\rm{O}}_2}}^{sys} + 2n_{{\rm{F}}{{\rm{e}}_{\rm{2}}}{\rm{Si}}{{\rm{O}}_4}}^{liq} - 2n_{{\rm{F}}{{\rm{e}}_2}{{\rm{O}}_3}}^{liq} - 2n_{{\rm{Si}}{{\rm{O}}_2}}^{liq}} \right)  - {\lambda _{{{\rm{H}}_{\rm{2}}}{\rm{O,}}n}}\left( {n_{{{\rm{H}}_2}{\rm{O}}}^{sys} - n_{{{\rm{H}}_2}{\rm{O}}}^{liq}} \right) 
\end{array}
$$

In [ ]:
def Lagrangian(nFe2O3in=0.000777, nH2Oin=0.183006, t=1053.15, p=1750.0):
    nLiq = setMoles(nFe2O3in, nH2Oin)
    nFix = np.empty((f,1))
    nFix[0] = 2.0*nLiq[1]+2.0*nLiq[0]-2.0*nLiq[2]
    nFix[1] = nLiq[3]
    n = np.vstack((nLiq,nFix))
    Gliq = liquid.getGibbsFreeEnergyFromMolesOfComponents_andT_andP_(m,t,p)
    muLiq = liquid.getChemicalPotentialFromMolesOfComponents_andT_andP_(m, t, p)
    result = Gliq - nFix[0]*muO2 - nFix[1]*muH2O
    # now add the Lagrange terms
    temp = np.matmul(Cproj, n)
    temp = np.subtract(temp, bcproj)
    result -= (np.dot(temp.transpose(), xLambda[0:2])[0][0])
    muLiqSiO2    = muLiq.valueAtIndex_(0)
    muLiqFe2O3   = muLiq.valueAtIndex_(3)
    muLiqFe2SiO4 = muLiq.valueAtIndex_(5)
    muLiqH2O     = muLiq.valueAtIndex_(14)
    result -= xLambda[2][0]*(muH2O-muLiqH2O) 
    result -= xLambda[3][0]*(muO2-(2.0*muLiqFe2O3+2.0*muLiqSiO2-2.0*muLiqFe2SiO4))
    result -= xLambda[4][0]*(nFix[0]-(2.0*nLiq[1]+2.0*nLiq[0]-2.0*nLiq[2]))
    result -= xLambda[5][0]*(nFix[1]-nLiq[3])
    return result

### Define the Wronskian of the Lagrangian function

In [ ]:
def Wronskian(nFe2O3in=0.000777, nH2Oin=0.183006, t=1053.15, p=1750.0):
    nLiq = setMoles(nFe2O3in, nH2Oin)
    w = np.zeros((c+f,c+f))
    
    d2gdm2 = liquid.getD2gDm2FromMolesOfComponents_andT_andP_(m, t, p)
    w[0][0] = d2gdm2.valueAtRowIndex_andColIndex_( 0,  0) # liq SiO2,    SiO2
    w[0][1] = d2gdm2.valueAtRowIndex_andColIndex_( 0,  3) # liq SiO2,    Fe2O3
    w[0][2] = d2gdm2.valueAtRowIndex_andColIndex_( 0,  5) # liq SiO2,    Fe2SiO4
    w[0][3] = d2gdm2.valueAtRowIndex_andColIndex_( 0, 14) # liq SiO2,    H2O
    w[1][1] = d2gdm2.valueAtRowIndex_andColIndex_( 3,  3) # liq Fe2O3,   Fe2O3
    w[1][2] = d2gdm2.valueAtRowIndex_andColIndex_( 3,  5) # liq Fe2O3,   Fe2SiO4
    w[1][3] = d2gdm2.valueAtRowIndex_andColIndex_( 3, 14) # liq Fe2O3,   H2O
    w[2][2] = d2gdm2.valueAtRowIndex_andColIndex_( 5,  5) # liq Fe2SiO4, Fe2SiO4
    w[2][3] = d2gdm2.valueAtRowIndex_andColIndex_( 5, 14) # liq Fe2SiO4, H2O
    w[3][3] = d2gdm2.valueAtRowIndex_andColIndex_(14, 14) # liq H2O,     H2O
    
    d3gdm3 = liquid.getD3gDm3FromMolesOfComponents_andT_andP_(m, t, p)
    w[0][0] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14, 0, 0)
    w[0][1] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14, 0, 3)
    w[0][2] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14, 0, 5)
    w[0][3] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14, 0,14)
    w[1][1] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14, 3, 3)
    w[1][2] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14, 3, 5)
    w[1][3] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14, 3,14)
    w[2][2] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14, 5, 5)
    w[2][3] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14, 5,14)
    w[3][3] += xLambda[2][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_(14,14,14)

    w[0][0] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0, 0, 0)
    w[0][1] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0, 0, 3)
    w[0][2] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0, 0, 5)
    w[0][3] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0, 0,14)
    w[1][1] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0, 3, 3)
    w[1][2] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0, 3, 5)
    w[1][3] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0, 3,14)
    w[2][2] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0, 5, 5)
    w[2][3] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0, 5,14)
    w[3][3] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 0,14,14)

    w[0][0] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3, 0, 0)
    w[0][1] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3, 0, 3)
    w[0][2] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3, 0, 5)
    w[0][3] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3, 0,14)
    w[1][1] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3, 3, 3)
    w[1][2] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3, 3, 5)
    w[1][3] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3, 3,14)
    w[2][2] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3, 5, 5)
    w[2][3] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3, 5,14)
    w[3][3] += 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 3,14,14)

    w[0][0] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5, 0, 0)
    w[0][1] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5, 0, 3)
    w[0][2] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5, 0, 5)
    w[0][3] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5, 0,14)
    w[1][1] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5, 3, 3)
    w[1][2] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5, 3, 5)
    w[1][3] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5, 3,14)
    w[2][2] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5, 5, 5)
    w[2][3] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5, 5,14)
    w[3][3] -= 2.0*xLambda[3][0]*d3gdm3.valueAtFirstIndex_andSecondIndex_andThirdIndex_( 5,14,14)

    for i in range(0,c+f):
        for j in range(i+1,c+f):
             w[j][i] = w[i][j]
    
    return w

### Form the QR decomposition of A
Note that A must be square, so zero filled rows are added 
$
{\bf{A}} = {\bf{QR}} = \left[ {\begin{array}{*{20}{c}}
{{{\bf{Q}}_1}}&{{{\bf{Q}}_2}}
\end{array}} \right]\left[ {\begin{array}{*{20}{c}}
{{{\bf{R}}_1}}\\
{\bf{0}}
\end{array}} \right]
$
and 
${{\bf{Q}}_2}^T$ is the matrix ${\bf{Z}}$

In [ ]:
A = Amatrix(nFe2O3in=nLiqFe2O3, nH2Oin=nLiqH2O)
#bottom = np.zeros((f,c+f))
#A = np.vstack((A, bottom))
#Q,R = lin.qr(A,mode='full')
R,Q = lin.rq(A,mode='full')
print ("Q matrix:")
for i in range(0,Q.shape[0]):
    if Q.shape[1] == c+f:
        print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e} {5:10.3e}".format( \
            Q[i][0], Q[i][1], Q[i][2], Q[i][3], Q[i][4], Q[i][5]))
    elif Q.shape[1] == c:
        print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e}".format( \
            Q[i][0], Q[i][1], Q[i][2], Q[i][3], Q[i][4]))
print ("R matrix:")
for i in range(0,R.shape[0]):
    if R.shape[1] == c+f:
        print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e} {5:10.3e}".format( \
            R[i][0], R[i][1], R[i][2], R[i][3], R[i][4], R[i][5]))
    elif R.shape[1] == c:
        print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e}".format( \
            R[i][0], R[i][1], R[i][2], R[i][3], R[i][4]))   
print ("Z matrix")
Z = Q[0:0,:].transpose()
for i in range(0,Z.shape[0]):
    if Z.shape[1] == f:
        print ("{0:10.3e} {1:10.3e}".format(Z[i][0], Z[i][1]))

### Form the vector of Lagrange multipliers
$$
{\bf{g}} = {{\bf{A}}^T}\left[ {\begin{array}{*{20}{c}}
{{\lambda _{{n_1}}}}\\
 \vdots \\
{{\lambda _{{n_{c - f}}}}}\\
{{\lambda _{{\phi _1}}}}\\
 \vdots \\
{{\lambda _{{\phi _f}}}}
\end{array}} \right]
$$

In [ ]:
print ("Gradient:")
g = Gradient(nFe2O3in=nLiqFe2O3, nH2Oin=nLiqH2O)
for i in range(0,g.shape[0]):
    print ("{0:10.3e}".format(g[i][0]))
print ("Lambda:")
xLambda = np.linalg.lstsq(A.transpose(),g)[0]
for i in range(0,xLambda.shape[0]):
    print ("{0:10.3e}".format(xLambda[i][0]))

### Compute the Wronskian of the Lagrangian

In [ ]:
W = Wronskian(nFe2O3in=nLiqFe2O3, nH2Oin=nLiqH2O)
for i in range(0,W.shape[0]):
    print ("{0:10.3e} {1:10.3e} {2:10.3e} {3:10.3e} {4:10.3e} {5:10.3e}".format( \
        W[i][0], W[i][1], W[i][2], W[i][3], W[i][4], W[i][5]))

### Next project the gradient and the Wronskian
The search direction ($\Delta {\bf{n}}_{_\phi }^i$) is given by
$$
{{\bf{Z}}^T}{\bf{WZ}}\Delta {\bf{n}}_\phi ^i =  - {{\bf{Z}}^T}{\bf{g}}
$$
and ${\bf{n}}_\phi ^{i + 1} = \Delta {\bf{n}}_{_\phi }^i + {\bf{n}}_\phi ^i$ gives the quadratic approximation to the minimum, which must be adjusted to maintain the non-linear equality constraints
$$
{\bf{\tilde n}}_\phi ^{i + 1} = s\Delta {\bf{n}}_\phi ^i + {\bf{n}}_\phi ^i + \Delta {\bf{n}}_\phi ^{corr}
$$

In [ ]:
ZTg = np.matmul(Z.transpose(),g)
ZTWZ = np.matmul(Z.transpose(), np.matmul(W,Z))
print("Z^T g")
for i in range(0,ZTg.shape[0]):
    print ("{0:10.3e}".format(ZTg[i][0]))
print("Z^T W Z")
for i in range(0,ZTWZ.shape[0]):
    print ("{0:10.3e} {1:10.3e}".format(ZTWZ[i][0], ZTWZ[i][1]))

### ... and solve for the correction vector

In [ ]:
x = lin.solve(ZTWZ, ZTg)
print ("Solution in the null space:")
for i in range(0,x.shape[0]):
    print ("{0:10.3e}".format(x[i][0]))
deltan = np.matmul(Z,x)
print ("Inflated Solution:")
for i in range(0,deltan.shape[0]):
    print ("{0:10.3e}".format(deltan[i][0]))

## Final solution
The correction vector is zero because the initial guess satisfies all constraints and the number of constraints is equal to the number of unknowns. 

In [ ]:
nLiq = setMoles(nFe2O3in=nLiqFe2O3, nH2Oin=nLiqH2O)
print ("n SiO2    liq = {0:8.6f}".format(nLiq[0][0]+deltan[0][0]))
print ("n Fe2O3   liq = {0:8.6f}".format(nLiq[1][0]+deltan[1][0]))
print ("n Fe2SiO4 liq = {0:8.6f}".format(nLiq[2][0]+deltan[2][0]))
print ("n H2O     liq = {0:8.6f}".format(nLiq[3][0]+deltan[3][0]))